In [46]:
import math
from random import randint, seed
from time import time, process_time

strchr = lambda x: chr(x)
strbyt = lambda x, y=0: ord(x[y])
bitlst = lambda x, y: x << y
bitrst = lambda x, y: x >> y
bitext = lambda x, y, z=1: bitrst(x, y) & int(math.pow(2, z) - 1)
bitxor = lambda x, y: x ^ y
bitbor = lambda x, y: x | y
btest  = lambda x, y: (x & y) != 0
mthrnd = lambda x, y: randint(x, y)
mthrsd = lambda x: seed(x)
osltim = lambda: int(time())
oslclk = lambda: process_time()


FL_NEGATE = bitlst(1, 1) # 2
FL_UNUSED3 = bitlst(1, 2) # 4
FL_XORBY6B = bitlst(1, 3) # 8
FL_XORBY3E = bitlst(1, 4) # 16
FL_UNUSED2 = bitlst(1, 5) # 32
FL_SWAPBYTES = bitlst(1, 6) # 64
FL_UNUSED1 = bitlst(1, 7) # 128

currtime = osltim()
while True:
    if osltim() - currtime != 0: # wait for next second
        break

mthrsd(osltim() + oslclk() * 1000) # initialize seed

def ValidateChar(char):
    if type(char) is str and len(char) == 1:
        char = strbyt(char)
    return char

def GenerateFlag():
    finalflag = 0
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_SWAPBYTES)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_NEGATE)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_XORBY6B)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_XORBY3E)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_UNUSED3)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_UNUSED2)
    if mthrnd(0, 1) == 1:
        finalflag = bitbor(finalflag, FL_UNUSED1)

    return finalflag # "random" int between 0 and 255

def CheckFlag(f, flag):
    return btest(f, flag) # (f & flag) != 0

def ESwapChar(char):
    char = ValidateChar(char)
    THIS_MSB = bitext(char, 4, 4)
    THIS_LSB = bitext(char, 0, 4)

    return strchr(bitbor(bitxor(THIS_MSB, 0x0D), bitxor(bitlst(THIS_LSB, 4), 0xB0)))

def XorBy6B(char):
    char = ValidateChar(char)
    
    return strchr(bitxor(char, 0x6B))

def XorBy3E(char):
    char = ValidateChar(char)
    
    return strchr(bitxor(char, 0x3E))

def NegateChar(char):
    char = ValidateChar(char)
    
    return strchr(255 - char)

FLAGS = []
CHARS = []

def AppendFlag(flag):
    FLAGS.append(strchr(bitxor(flag, 0x4A)))

def EncryptCharacter(char):
    char = ValidateChar(char)
    flag = GenerateFlag()

    if CheckFlag(flag, FL_SWAPBYTES):
        char = ESwapChar(char)
    if CheckFlag(flag, FL_NEGATE):
        char = NegateChar(char)
    if CheckFlag(flag, FL_XORBY6B):
        char = XorBy6B(char)
    if CheckFlag(flag, FL_XORBY3E):
        char = XorBy3E(char)

    return char, flag

def _Encrypt(string):
    for i in range(0, len(string)):
        char, flag = EncryptCharacter(strbyt(string, i)) # send int to EncryptCharacter

        if type(char) is int:
            char = strchr(char)

        CHARS.append(char) # append a real char
        AppendFlag(flag) # append an int

def Encrypt(string):
    _Encrypt(string)

    output = [f"{str(ord(v))} {str(ord(FLAGS[i]))}" for i, v in enumerate(CHARS)]
    file = open("output.txt", "w")
    file.write(' '.join(output))
    file.close()

In [174]:
out = "108 182 82 176 167 158 69 222 39 102 234 14 241 16 10 218 160 108 76 234 225 224 1 12 97 122 114 90 10 90 250 14 155 80 101 186 97 218 115 218 207 76 190 174 196 84 192 144"

In [175]:
outlist = out.split(" ")

In [176]:
ct = []
for (i, n) in enumerate(outlist):
    if i % 2 == 0:
        ct.append((n, outlist[i+1]))

In [178]:
print(ct)

[('108', '182'), ('82', '176'), ('167', '158'), ('69', '222'), ('39', '102'), ('234', '14'), ('241', '16'), ('10', '218'), ('160', '108'), ('76', '234'), ('225', '224'), ('1', '12'), ('97', '122'), ('114', '90'), ('10', '90'), ('250', '14'), ('155', '80'), ('101', '186'), ('97', '218'), ('115', '218'), ('207', '76'), ('190', '174'), ('196', '84'), ('192', '144')]


In [75]:
[ord(ESwapChar(ord(ESwapChar(ord(ESwapChar(ord(ESwapChar(_)))))))) for _ in range(256)] == [_ for _ in range(256)]

True

$($ord $\circ$ ESwapChar$)^4 = \mathrm{id}$

In [79]:
[ord(NegateChar(ord(NegateChar(_)))) for _ in range(256)] == [_ for _ in range(256)]

True

$($ord $\circ$ NegateChar$)^2 = \mathrm{id}$

In [81]:
[ord(XorBy6B(ord(XorBy6B(_)))) for _ in range(256)] == [ord(XorBy3E(ord(XorBy3E(_)))) for _ in range(256)] == [_ for _ in range(256)]

True

$($ord $\circ$ XorBy6B$)^2 = ($ord $\circ$ XorBy3E$)^2 = \mathrm{id}$

In [305]:
def DecryptCharacter(t): # flag must be int
    (char, flag) = t
    char = ValidateChar(char)

    if CheckFlag(flag, FL_XORBY3E):
        char = XorBy3E(char)
    if CheckFlag(flag, FL_XORBY6B):
        char = XorBy6B(char)
    if CheckFlag(flag, FL_NEGATE):
        char = NegateChar(char)
    if CheckFlag(flag, FL_SWAPBYTES):
        char = ESwapChar(ValidateChar(ESwapChar(ValidateChar(ESwapChar(char)))))
        
    if type(char) is int:
        char = strchr(char)

    return char

In [359]:
pt = []
for (c, f) in ct:
    pt.append(DecryptCharacter((ValidateChar(int(c)), bitxor(int(f), 0x4A))))
print("".join(pt))

HTB{Lun4_Lu4_L4t1n_M00n}
